**Łukasz Folwarczyk**

*"Badanie i implementacja algorytmów detekcji drogi przy wykorzystaniu cyfrowego przetwarzania obrazów."*

---

# Videos to images


Notatnik implementujący funkcję pozwalająca na wyłuskanie z materiału wideo pojedynczych klatek (zdjęć) wykorzystanych do procesu uczenia głebokiej sieci neuronowej. Działanie modułu opisane zostało w paragrafie 4.3.




In [ ]:
import os
import cv2
import subprocess
import argparse
import time

In [ ]:
def video2images(input_dir, output_dir, video_exts, out_shape = None):
    """ 
    Opis:
    ---
    Funkcja konwertuje materiał wideo, zlokalizowany w katalogu input_dir 
    na serię zdjęć i zapisuje w katalogu o nazwie takiej jak nazwa filmu 
    w katalogu output_dir. Jeżeli w katalogu input_dir znajduję się n 
    materiałów wideo, wówczas w katalogu output_dir powstanie n katalogów, 
    ze zdjęciami zwiazanymi z danym materiałem wideo. Zdjęcia zapisywane 
    są w formacje jpg.    
    
    
    Argumenty:
    ---
        input_dir (string):  nazwa katalogu wejściowego, 
                             zawierającego materiał wideo
                            
        output_dir (string): nazwa katalogu wyjściowego,
                             w którym zapisane zostaną zdjęcia

        video_exts (string): rozszerzenie materiału wideo
        
        out_shape (tuple):   rozmiar zdjęcia, jeżli None 
                             to równy rozmiarowi wideo


    Return:
    ---
        None
    """
    
    if not os.path.exists(output_dir):               # stworzenie katalogu wyjściowego (jeśli nie istnieje)
        os.makedirs(output_dir)

    videos = os.listdir(input_dir)                   # wczytanie nazw plików wideo z katalogu wejściowego
    video_counter = 0
    imgs_counter = 0

    # pobranie i zapisanie klatek pochodzących z każdego materiału wideo
    for video in videos:
        # klatki zapisywane są w katalogo o nazwie takiej samej jak nazwa pliku wideo (bez rozszerzenia)
        if output_dir[-1] != '/':
            img_out_dir = output_dir + '/' + (video[:-len(video_exts)]) + '/'
        else:
            img_out_dir = output_dir + (video[:-len(video_exts)])  + '/'
        if not os.path.exists(img_out_dir):
            os.makedirs(img_out_dir)

        if video.lower().endswith(video_exts):
            # wyznaczenie pełnej ścieżki do materiału wideo
            if input_dir[-1] != '/':
                input_video_file = input_dir + '/' + video
            else:
                input_video_file = input_dir + video
            
            # nazwa klatki = nazwa materiału wideo + numer pobranej klatki    
            new_image_name = (video[:-len(video_exts)]) + "-{0}.jpg"

            print("Konwersja wideo na zdjęcia...")
            print("Plik wideo: {0}".format(input_video_file))
            print("Powiązany katalog zdjęć: {0}".format(img_out_dir))
            
            # odczytanie materiału wideo
            vidcap = cv2.VideoCapture(input_video_file)
            
            # próba pobrania 1 klatki
            success,image = vidcap.read()
            counter1 = 0
            counter2 = 0
            
            # jeśli próba pobrania klatki sie powiodła
            while success:
                if counter2 == 20:                                     # pobranie co 20 klatki    
                    counter2 = 0                                       # (decyduje o ilość powstałych zdjęć)
                    new_img_path = img_out_dir + new_image_name        # ścieżka + nazwa klatki 
                    if out_shape != None:
                        image = cv2.resize(image, (out_shape))
                    cv2.imwrite(new_img_path.format(counter1), image)  # zapisanie klatki 
                    imgs_counter += 1
                    counter1 += 1
                counter2 += 1
                success,image = vidcap.read()                          # pobranie kolejnej klatki
                
            video_counter += 1
            print()

    print("Liczba przekonwertowanych wideo: {0}".format(video_counter))
    print("Liczba powstałych zdjęć: {0}".format(imgs_counter))

# Przykład

In [ ]:
input_dir = r"/media/lukasz/Dysk lokalny/MgrPliki/Dataset/park_filmy"
output_dir = r"/media/lukasz/Dysk lokalny/MgrPliki/Dataset/example"
video_exts = ".mp4"

In [ ]:
video2images(input_dir, output_dir, video_exts)